## Implementierung des HybridRAG-Systems

In [3]:
import os
import pandas as pd


from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from neo4j import GraphDatabase
from langchain_community.document_loaders import TextLoader
from langchain_neo4j import Neo4jGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore



load_dotenv()

True

## GraphRAG-System erstellen

#### Verbindung zum knowledge Graphen herstellen

In [2]:
uri = os.getenv("NEO4J_URI")  
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")

In [3]:
graph = Neo4jGraph(url=uri, username=username, password=password)

### LLM Instanzieren

In [10]:
llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=os.environ["OPEN_AI_API_KEY"]
)

### Fulltext index wird erstellt

In [5]:
# Der folgende Code stammt aus dem Repository: GraphRAG-with-Llama-3.1

# Quelle: https://github.com/Coding-Crashkurse/GraphRAG-with-Llama-3.1/blob/main/enhancing_rag_with_graph.ipynb


driver = GraphDatabase.driver(
        uri = uri,
        auth = (username, password))

def create_fulltext_index(tx):
    query = '''
    CREATE FULLTEXT INDEX `fulltext_entity_id` 
    FOR (n:__Entity__) 
    ON EACH [n.id];
    '''
    tx.run(query)

# Function to execute the query
def create_index():
    with driver.session() as session:
        session.execute_write(create_fulltext_index)
        print("Fulltext index created successfully.")

# Call the function to create the index
try:
    create_index()
except:
    pass

# Close the driver connection
driver.close()

### Extrahiert die Entities aus der Frage

In [8]:
with open("../prompts/entity_chain_prompt.txt", "r", encoding="utf-8") as e:
    entity_prompt = e.read()

In [11]:
prompt_extrahiere_entitaet = ChatPromptTemplate.from_template(
f"{entity_prompt}"
)

entity_chain = (
    {"frage": RunnablePassthrough()}
    | prompt_extrahiere_entitaet
    | llm
    | StrOutputParser()
    | (lambda x: [name.strip() for name in x.split(",")])
)


In [7]:
# Der folgende Code ist eine Anlehnung an den Code aus dem Repository: GraphRAG-with-Llama-3.1

# Quelle: https://github.com/Coding-Crashkurse/GraphRAG-with-Llama-3.1/blob/main/enhancing_rag_with_graph.ipynb

def graph_retriever(question: str):
    """
    Gibt die Triplets zurück

    """
    result = ""
    # Entitäten aus der Frage Extrahieren
    entities = entity_chain.invoke(question)
    # Über die Entitäten iterieren
    for entity in entities:
        # Cypher Abfrage an den Knowledge Graph, dass die ausgehnden und eingehenden Relationen gefunden werden und als Triplet zurückgegben werden 
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('fulltext_entity_id', $query, {limit:2})
                YIELD node, score
                CALL {
                WITH node
                MATCH (node)-[r]->(neighbor)
                WHERE type(r) <> 'HAS_ENTITY'
                RETURN DISTINCT node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
                UNION ALL
                WITH node
                MATCH (node)<-[r]-(neighbor)
                WHERE type(r) <> 'HAS_ENTITY'
                RETURN DISTINCT neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
                }
                RETURN output LIMIT 50
    
            """,
            {"query": entity},
        )
        result += "\n".join([el['output'] for el in response])
    return result

## VectorRAG-System implementieren

#### Artikel einlesen

In [8]:


# Pfad zum Ordner mit manipulierten oder Original Artikeln 
ordnerpfad = r"C:\bachelor\experiment_bachelor_lb\Datenmanipulation\originalartikel" # Pfad ändern zu den manipulierten Artikeln ohne markierung

# liste mit dem extrahierten Text aus den Artikeln
alle_docs = []

# Über den Ordner Iterieren
for filename in os.listdir(ordnerpfad):
    # nur txt Dateien
    if filename.endswith(".txt"):
        # vollständiger Pfad
        file_path = os.path.join(ordnerpfad, filename)
        # TextLoader instanzieren
        loader = TextLoader(file_path=file_path, encoding="utf8")
        # Text extrahieren
        docs = loader.load()
        # Text zur Liste hinzufügen,(for) weil der Loader eine Liste zurückgibt
        for doc in docs:
            alle_docs.append(doc)

print(f"{len(alle_docs)} Dokumente erfolgreich geladen")


95 Dokumente erfolgreich geladen und bereinigt.


#### Eingelesenen Dokumente Chunken

In [9]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
documents = text_splitter.split_documents(documents=alle_docs)

#### Embeddings Instanzieren

In [20]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=os.environ["OPEN_AI_API_KEY"])

#### Vectorstore instanzieren und Chunks hinzufügen

In [11]:

vector_store = InMemoryVectorStore(embeddings)

In [12]:
vector_store.add_documents(documents=documents)

['fcb5e299-f982-44ac-8750-2876e0dd1230',
 'bf96d6a3-96f9-4276-834e-bb44d4ccecd9',
 '326c841a-33a0-40d8-bc40-9a4eb2277447',
 'd2369273-0b5e-4de6-ba44-4036932ad2bf',
 '4bf9836a-44fb-4ea1-bd20-4e20955f4298',
 'af31c9ca-72d0-4ea6-b3d9-c8ecd0944de1',
 'e0c6cc00-68ca-48bd-afe9-8cb761f875bb',
 '3ff92a13-26e9-462b-8f82-9f4d939a0fc0',
 'd09b9d9f-5bca-404e-826f-ec0b8fb094ea',
 '88909f72-33f6-4e1f-b46f-39e0e2f14d67',
 'd775bf58-efe0-4669-aaac-ac42c32d335b',
 'f146a13b-5679-4e8c-b35d-d34dd9c17c0f',
 '16e08b4c-68ab-41d9-b02a-fa22148be4a5',
 '57d70c5b-b1e0-48ae-9be5-b45608e1b201',
 '03435405-fc2e-4660-9755-2a54f07f5ab9',
 'cdf4b525-c3b1-4899-ba6d-354035656a33',
 '408fa3f1-6d1c-48e5-88cb-ba85792f297a',
 '745a1ee1-e0c3-456f-80d0-c94a341ea64d',
 'edb4f7fd-f93e-4312-a0a9-b671632b1f32',
 '5f3ace76-611e-4cc7-993c-387591435a11',
 '3d397592-4c5a-455d-a2d5-f23f77f201db',
 '831098fe-cae5-4e53-93fb-50fad39d652f',
 'fbf65fc0-2833-4bc1-92bd-d9281e6b3e4e',
 '1951d2f7-6b1e-4831-ac11-7bc02be11188',
 '14590631-8a3e-

#### Retrieve Funktion

In [ ]:
def retrieve(query):
    """extrahiert den Text aus den
     Document Chunks"""
    # 3 ähnlichsten chunks zur query zurückgeben
    abgerufene_dokumente = vector_store.similarity_search(query, k=3)
    # Formatieren, dass nur der Inhalt zurückgeben wird
    formatiert = "\n\n\n".join(
        (f"Content: {doc.page_content}")
        for doc in abgerufene_dokumente
    )
    return formatiert

## Vollständige Retrieve Funktion die Beide Kontexte konkateniert

In [ ]:
def full_retriever(question):
    """Generiert den vollständigen Kontext für eine gegebene Frage, 
    indem Graph- und Vektordaten kombiniert werden."""
    
    # Hole die Graph-Daten und Vektor-Daten
    graph_data = graph_retriever(question)  
    vector_data = retrieve(question)


    # Falls keine Graph-Daten existieren, eine Platzhalter-Nachricht setzen
    graph_data_final = f"### Graph Data:\n{graph_data}" if graph_data else "### Graph Data:\n(No relevant graph data found.)"

   

    # Kombinierter, formatierter Kontext
    final_data = f"""
        {graph_data_final}

        ### Kontext aus der Vektordatenbank:

        {vector_data}


    """.strip()  


    return final_data


#### Prompt für das LLM

In [1]:
with open("../prompts/hybrid_rag.txt", "r", encoding="utf-8") as h:
    hybrid_prompt = h.read()

In [6]:

template = f"{hybrid_prompt}"

prompt = ChatPromptTemplate.from_template(template)

#### Chain für das HybridRAG-System erstellen

In [ ]:


chain_hybrid_rag = (
    {
        "context": full_retriever,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

#### Funktion zur Beantwortung aller Fragen

In [ ]:


def run_hybrid_rag_antworten(csv_path, save_path):
    """ 
    Die Fragen werden dem System gestellt, die Antworten werden 
    generiert und als neue Spalte zu der CSV Datei hinzugefügt
    """
    # CSV mit den Fragen laden, die Antworten der anderen beiden Systeme sind in der CSV ebenso enthalten
    df = pd.read_csv(csv_path)

    # Spalte für Antworten vorbereiten
    antworten = []
    # Über die Zeilen des Dfs iterieren
    for _, row in df.iterrows():
    # Die Frage aus der jeweiligen Zeile Extrahieren
        frage = row["frage"]
        try:
            # Chain wird mit der Frage als Parameter aufgerufen
            antwort = chain_hybrid_rag.invoke(frage)
        except Exception as e:
            print(f"Fehler bei Artikel {row['artikel_id']} Frage {row['frage_nr']}: {e}")
            antwort = "FEHLER"
        # Antwort zur liste hinzufügen
        antworten.append(antwort)
    # Spalte mit den Antworten des Systems zum Dataframe hinzufügen
    df["hybrid_rag_antwort_original_2"] = antworten # Spalte je nach durchgang ändern

    # DF als CSV speichern (am besten noch auslagern)
    if save_path:
        df.to_csv(save_path, index=False, encoding="utf-8")
        print(f"Ergebnisse gespeichert unter: {save_path}")

    return df

In [ ]:
df_hybridRAG_antworten = run_hybrid_rag_antworten("../antworten_der_systeme_originalkontext_beide_durchgänge/graph_rag_und_vector_rag_antworten_original_2.csv",
                         "../antworten_der_systeme_originalkontext_beide_durchgänge/graph_rag_vector_rag_hybrid_rag_antworten_original_2.csv" )